In [1]:
# Import libraries for data wrangling, preprocessing and visualization
import numpy 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
# Importing libraries for building the neural network
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.
/Users/shinde/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/shinde/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/shinde/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/shinde/anaconda3/lib/python3.7/site-packages/t

In [14]:
# Read data file
data = pd.read_csv("/Users/shinde/Desktop/breast_data.csv", header=0)
seed = 5
numpy.random.seed(seed)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
id                         569 non-null int64
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non

In [16]:
# Column Unnamed : 32 holds only null values, so it is of no use to us. We simply drop that column.
data.drop("Unnamed: 32",axis=1,inplace=True)
data.drop("id", axis=1, inplace=True)

In [17]:
data.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [18]:
data.head(2)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [20]:
#Created just for correlation metrics analysis: Correlation Study
data['diagnosis'].unique()

array(['M', 'B'], dtype=object)

In [30]:
#Function to convert M & B to 1 & 0
def encoded_diagnosis(df):
    if df['diagnosis'] == 'M':
        val = 1
    else:
        val = 0
    return val

In [31]:
#Apply encoded_diagnosis function on df 
data['diagnosis_test'] = data.apply(encoded_diagnosis, axis = 1 )


In [32]:
data.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'diagnosis_test'],
      dtype='object')

In [36]:
data.shape

(569, 32)

In [35]:
#Observation 1: Highly correlated features are KEPT in model (Multi-collinearity automatically handled by NN)
#Onservation 2: Features less correlated with Y are still klept in model (NN handles automatically)
data.corr()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_test
radius_mean,1.000000,0.323782,0.997855,0.987357,0.170581,0.506124,0.676764,0.822529,0.147741,-0.311631,...,0.297008,0.965137,0.941082,0.119616,0.413463,0.526911,0.744214,0.163953,0.007066,0.730029
texture_mean,0.323782,1.000000,0.329533,0.321086,-0.023389,0.236702,0.302418,0.293464,0.071401,-0.076437,...,0.912045,0.358040,0.343546,0.077503,0.277830,0.301025,0.295316,0.105008,0.119205,0.415185
perimeter_mean,0.997855,0.329533,1.000000,0.986507,0.207278,0.556936,0.716136,0.850977,0.183027,-0.261477,...,0.303038,0.970387,0.941550,0.150549,0.455774,0.563879,0.771241,0.189115,0.051019,0.742636
area_mean,0.987357,0.321086,0.986507,1.000000,0.177028,0.498502,0.685983,0.823269,0.151293,-0.283110,...,0.287489,0.959120,0.959213,0.123523,0.390410,0.512606,0.722017,0.143570,0.003738,0.708984
smoothness_mean,0.170581,-0.023389,0.207278,0.177028,1.000000,0.659123,0.521984,0.553695,0.557775,0.584792,...,0.036072,0.238853,0.206718,0.805324,0.472468,0.434926,0.503053,0.394309,0.499316,0.358560
compactness_mean,0.506124,0.236702,0.556936,0.498502,0.659123,1.000000,0.883121,0.831135,0.602641,0.565369,...,0.248133,0.590210,0.509604,0.565541,0.865809,0.816275,0.815573,0.510223,0.687382,0.596534
concavity_mean,0.676764,0.302418,0.716136,0.685983,0.521984,0.883121,1.000000,0.921391,0.500667,0.336783,...,0.299879,0.729565,0.675987,0.448822,0.754968,0.884103,0.861323,0.409464,0.514930,0.696360
concave points_mean,0.822529,0.293464,0.850977,0.823269,0.553695,0.831135,0.921391,1.000000,0.462497,0.166917,...,0.292752,0.855923,0.809630,0.452753,0.667454,0.752399,0.910155,0.375744,0.368661,0.776614
symmetry_mean,0.147741,0.071401,0.183027,0.151293,0.557775,0.602641,0.500667,0.462497,1.000000,0.479921,...,0.090651,0.219169,0.177193,0.426675,0.473200,0.433721,0.430297,0.699826,0.438413,0.330499
fractal_dimension_mean,-0.311631,-0.076437,-0.261477,-0.283110,0.584792,0.565369,0.336783,0.166917,0.479921,1.000000,...,-0.051269,-0.205151,-0.231854,0.504942,0.458798,0.346234,0.175325,0.334019,0.767297,-0.012838


In [29]:
#Just for correlation
del data['diagnosis_test'] 

In [41]:
data.dtypes

diagnosis                   object
radius_mean                float64
texture_mean               float64
perimeter_mean             float64
area_mean                  float64
smoothness_mean            float64
compactness_mean           float64
concavity_mean             float64
concave points_mean        float64
symmetry_mean              float64
fractal_dimension_mean     float64
radius_se                  float64
texture_se                 float64
perimeter_se               float64
area_se                    float64
smoothness_se              float64
compactness_se             float64
concavity_se               float64
concave points_se          float64
symmetry_se                float64
fractal_dimension_se       float64
radius_worst               float64
texture_worst              float64
perimeter_worst            float64
area_worst                 float64
smoothness_worst           float64
compactness_worst          float64
concavity_worst            float64
concave points_worst

In [40]:
data.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'diagnosis_test'],
      dtype='object')

In [42]:
# Select the columns to use for prediction in the neural network
prediction_var = ['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']

X = data[prediction_var].values
Y = data.diagnosis.values #Y is M & B

In [43]:
# Diagnosis values are strings. Changing them into numerical values using LabelEncoder.
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [52]:
# Baseline model for the neural network. 
# We choose a hidden layer of 10 neurons. 
# The lesser number of neurons helps to "eliminate the redundancies in the data" and select the more important features.
# Input Features = 30
# Activation Function is Sigmmoid: due to classification 
# Accuarcy : Classification Accuracy

def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(10, input_dim=30, kernel_initializer='normal', activation='relu')) #layer 1
    #model.add(Dense(5, kernel_initializer='normal', activation='relu')) #layer 2
    # Accuracy dropped from 1.93% to 1.87 after adding 1 extra hidden layer
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid')) #final layer 3 hence sigmoid
    # Compile model
    # We use the the logarithmic loss function, and the Adam gradient optimizer.
    # Try different optimizers than Adam - still a gradient descent
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [53]:
# Evaluate model using standardized dataset. 
estimators = []
estimators.append(('standardize', StandardScaler())) #All column values on same scale
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True) #stratified sampling to make group sless similar
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

#Dying kernal problem: pip install -U numpy  & conda clean -tipsy

Epoch 1/100
511/511 [==============================] - 2s 4ms/step - loss: 0.5945 - acc: 0.8806
Epoch 2/100
511/511 [==============================] - 0s 364us/step - loss: 0.2652 - acc: 0.9491
Epoch 3/100
511/511 [==============================] - 0s 358us/step - loss: 0.1467 - acc: 0.9648
Epoch 4/100
511/511 [==============================] - 0s 352us/step - loss: 0.1095 - acc: 0.9687
Epoch 5/100
511/511 [==============================] - 0s 358us/step - loss: 0.0898 - acc: 0.9746
Epoch 6/100
511/511 [==============================] - 0s 373us/step - loss: 0.0786 - acc: 0.9785
Epoch 7/100
511/511 [==============================] - 0s 390us/step - loss: 0.0710 - acc: 0.9785
Epoch 8/100
511/511 [==============================] - 0s 365us/step - loss: 0.0654 - acc: 0.9804
Epoch 9/100
511/511 [==============================] - 0s 344us/step - loss: 0.0616 - acc: 0.9843
Epoch 10/100
511/511 [==============================] - 0s 373us/step - loss: 0.0580 - acc: 0.9863
Epoch 11/100
511/511 

511/511 [==============================] - 0s 356us/step - loss: 0.0097 - acc: 0.9961
Epoch 84/100
511/511 [==============================] - 0s 434us/step - loss: 0.0092 - acc: 0.9980
Epoch 85/100
511/511 [==============================] - 0s 340us/step - loss: 0.0093 - acc: 0.9980
Epoch 86/100
511/511 [==============================] - 0s 439us/step - loss: 0.0088 - acc: 0.9980
Epoch 87/100
511/511 [==============================] - 0s 436us/step - loss: 0.0085 - acc: 0.9980
Epoch 88/100
511/511 [==============================] - 0s 503us/step - loss: 0.0083 - acc: 0.9980
Epoch 89/100
511/511 [==============================] - 0s 609us/step - loss: 0.0083 - acc: 0.9980
Epoch 90/100
511/511 [==============================] - 0s 409us/step - loss: 0.0079 - acc: 0.9980
Epoch 91/100
511/511 [==============================] - 0s 562us/step - loss: 0.0080 - acc: 0.9961
Epoch 92/100
511/511 [==============================] - 0s 543us/step - loss: 0.0083 - acc: 0.9980
Epoch 93/100
511/511 [=

511/511 [==============================] - 0s 358us/step - loss: 0.0251 - acc: 0.9922
Epoch 66/100
511/511 [==============================] - 0s 344us/step - loss: 0.0246 - acc: 0.9922
Epoch 67/100
511/511 [==============================] - 0s 348us/step - loss: 0.0244 - acc: 0.9922
Epoch 68/100
511/511 [==============================] - 0s 329us/step - loss: 0.0249 - acc: 0.9922
Epoch 69/100
511/511 [==============================] - 0s 370us/step - loss: 0.0238 - acc: 0.9922
Epoch 70/100
511/511 [==============================] - 0s 364us/step - loss: 0.0235 - acc: 0.9922
Epoch 71/100
511/511 [==============================] - 0s 406us/step - loss: 0.0235 - acc: 0.9922
Epoch 72/100
511/511 [==============================] - 0s 375us/step - loss: 0.0231 - acc: 0.9922
Epoch 73/100
511/511 [==============================] - 0s 356us/step - loss: 0.0226 - acc: 0.9922
Epoch 74/100
511/511 [==============================] - 0s 386us/step - loss: 0.0223 - acc: 0.9922
Epoch 75/100
511/511 [=

512/512 [==============================] - 0s 366us/step - loss: 0.0332 - acc: 0.9902
Epoch 48/100
512/512 [==============================] - 0s 335us/step - loss: 0.0329 - acc: 0.9922
Epoch 49/100
512/512 [==============================] - 0s 331us/step - loss: 0.0324 - acc: 0.9922
Epoch 50/100
512/512 [==============================] - 0s 336us/step - loss: 0.0320 - acc: 0.9922
Epoch 51/100
512/512 [==============================] - 0s 338us/step - loss: 0.0315 - acc: 0.9922
Epoch 52/100
512/512 [==============================] - 0s 349us/step - loss: 0.0312 - acc: 0.9922
Epoch 53/100
512/512 [==============================] - 0s 374us/step - loss: 0.0305 - acc: 0.9922
Epoch 54/100
512/512 [==============================] - 0s 327us/step - loss: 0.0301 - acc: 0.9922
Epoch 55/100
512/512 [==============================] - 0s 330us/step - loss: 0.0298 - acc: 0.9922
Epoch 56/100
512/512 [==============================] - 0s 333us/step - loss: 0.0293 - acc: 0.9922
Epoch 57/100
512/512 [=

512/512 [==============================] - 0s 351us/step - loss: 0.0438 - acc: 0.9883
Epoch 30/100
512/512 [==============================] - 0s 330us/step - loss: 0.0431 - acc: 0.9883
Epoch 31/100
512/512 [==============================] - 0s 338us/step - loss: 0.0426 - acc: 0.9863
Epoch 32/100
512/512 [==============================] - 0s 344us/step - loss: 0.0419 - acc: 0.9902
Epoch 33/100
512/512 [==============================] - 0s 377us/step - loss: 0.0410 - acc: 0.9883
Epoch 34/100
512/512 [==============================] - 0s 358us/step - loss: 0.0406 - acc: 0.9902
Epoch 35/100
512/512 [==============================] - 0s 336us/step - loss: 0.0396 - acc: 0.9902
Epoch 36/100
512/512 [==============================] - 0s 344us/step - loss: 0.0391 - acc: 0.9902
Epoch 37/100
512/512 [==============================] - 0s 334us/step - loss: 0.0386 - acc: 0.9902
Epoch 38/100
512/512 [==============================] - 0s 345us/step - loss: 0.0380 - acc: 0.9902
Epoch 39/100
512/512 [=

...

512/512 [==============================] - 0s 363us/step - loss: 0.0173 - acc: 0.9941
Epoch 58/100
512/512 [==============================] - 0s 370us/step - loss: 0.0164 - acc: 0.9941
Epoch 59/100
512/512 [==============================] - 0s 335us/step - loss: 0.0161 - acc: 0.9941
Epoch 60/100
512/512 [==============================] - 0s 368us/step - loss: 0.0159 - acc: 0.9941
Epoch 61/100
512/512 [==============================] - 0s 340us/step - loss: 0.0153 - acc: 0.9941
Epoch 62/100
512/512 [==============================] - 0s 368us/step - loss: 0.0150 - acc: 0.9941
Epoch 63/100
512/512 [==============================] - 0s 338us/step - loss: 0.0148 - acc: 0.9941
Epoch 64/100
512/512 [==============================] - 0s 350us/step - loss: 0.0145 - acc: 0.9941
Epoch 65/100
512/512 [==============================] - 0s 345us/step - loss: 0.0140 - acc: 0.9941
Epoch 66/100
512/512 [==============================] - 0s 344us/step - loss: 0.0138 - acc: 0.9941
Epoch 67/100
512/512 [=

513/513 [==============================] - 0s 361us/step - loss: 0.0345 - acc: 0.9922
Epoch 40/100
513/513 [==============================] - 0s 342us/step - loss: 0.0339 - acc: 0.9903
Epoch 41/100
513/513 [==============================] - 0s 368us/step - loss: 0.0331 - acc: 0.9903
Epoch 42/100
513/513 [==============================] - 0s 341us/step - loss: 0.0320 - acc: 0.9922
Epoch 43/100
513/513 [==============================] - 0s 344us/step - loss: 0.0319 - acc: 0.9903
Epoch 44/100
513/513 [==============================] - 0s 357us/step - loss: 0.0311 - acc: 0.9922
Epoch 45/100
513/513 [==============================] - 0s 349us/step - loss: 0.0304 - acc: 0.9922
Epoch 46/100
513/513 [==============================] - 0s 379us/step - loss: 0.0304 - acc: 0.9922
Epoch 47/100
513/513 [==============================] - 0s 365us/step - loss: 0.0291 - acc: 0.9922
Epoch 48/100
513/513 [==============================] - 0s 342us/step - loss: 0.0285 - acc: 0.9922
Epoch 49/100
513/513 [=

513/513 [==============================] - 0s 368us/step - loss: 0.0522 - acc: 0.9844
Epoch 22/100
513/513 [==============================] - 0s 343us/step - loss: 0.0509 - acc: 0.9844
Epoch 23/100
513/513 [==============================] - 0s 381us/step - loss: 0.0502 - acc: 0.9844
Epoch 24/100
513/513 [==============================] - 0s 341us/step - loss: 0.0493 - acc: 0.9864
Epoch 25/100
513/513 [==============================] - 0s 343us/step - loss: 0.0485 - acc: 0.9864
Epoch 26/100
513/513 [==============================] - 0s 344us/step - loss: 0.0477 - acc: 0.9864
Epoch 27/100
513/513 [==============================] - 0s 352us/step - loss: 0.0470 - acc: 0.9864
Epoch 28/100
513/513 [==============================] - 0s 346us/step - loss: 0.0463 - acc: 0.9864
Epoch 29/100
513/513 [==============================] - 0s 373us/step - loss: 0.0456 - acc: 0.9864
Epoch 30/100
513/513 [==============================] - 0s 350us/step - loss: 0.0449 - acc: 0.9864
Epoch 31/100
513/513 [=

513/513 [==============================] - 0s 390us/step - loss: 0.1543 - acc: 0.9532
Epoch 4/100
513/513 [==============================] - 0s 362us/step - loss: 0.1167 - acc: 0.9688
Epoch 5/100
513/513 [==============================] - 0s 357us/step - loss: 0.0978 - acc: 0.9708
Epoch 6/100
513/513 [==============================] - 0s 345us/step - loss: 0.0861 - acc: 0.9766
Epoch 7/100
513/513 [==============================] - 0s 345us/step - loss: 0.0785 - acc: 0.9786
Epoch 8/100
513/513 [==============================] - 0s 357us/step - loss: 0.0730 - acc: 0.9825
Epoch 9/100
513/513 [==============================] - 0s 342us/step - loss: 0.0690 - acc: 0.9844
Epoch 10/100
513/513 [==============================] - 0s 378us/step - loss: 0.0664 - acc: 0.9844
Epoch 11/100
513/513 [==============================] - 0s 343us/step - loss: 0.0635 - acc: 0.9844
Epoch 12/100
513/513 [==============================] - 0s 349us/step - loss: 0.0614 - acc: 0.9844
Epoch 13/100
513/513 [=======

513/513 [==============================] - 0s 394us/step - loss: 0.0123 - acc: 0.9942
Epoch 86/100
513/513 [==============================] - 0s 342us/step - loss: 0.0122 - acc: 0.9922
Epoch 87/100
513/513 [==============================] - 0s 343us/step - loss: 0.0124 - acc: 0.9961
Epoch 88/100
513/513 [==============================] - 0s 359us/step - loss: 0.0120 - acc: 0.9961
Epoch 89/100
513/513 [==============================] - 0s 349us/step - loss: 0.0116 - acc: 0.9961
Epoch 90/100
513/513 [==============================] - 0s 373us/step - loss: 0.0113 - acc: 0.9961
Epoch 91/100
513/513 [==============================] - 0s 351us/step - loss: 0.0113 - acc: 0.9961
Epoch 92/100
513/513 [==============================] - 0s 356us/step - loss: 0.0116 - acc: 0.9961
Epoch 93/100
513/513 [==============================] - 0s 367us/step - loss: 0.0112 - acc: 0.9961
Epoch 94/100
513/513 [==============================] - 0s 354us/step - loss: 0.0111 - acc: 0.9961
Epoch 95/100
513/513 [=

In [48]:
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
#Dont change OS variables if Kernal keep dying

Results: 97.90% (1.87%)


In [54]:
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
#Dont change OS variables if Kernal keep dying

Results: 97.71% (1.93%)


In [49]:
print(pipeline)

Pipeline(memory=None,
     steps=[('standardize', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', <keras.wrappers.scikit_learn.KerasClassifier object at 0x112bcb198>)])


In [50]:
print(results)

[1.         0.94827587 0.98245614 0.98245614 1.         0.94736842
 0.96491228 0.98214286 0.98214286 1.        ]


In [51]:
type(results)

numpy.ndarray

In [ ]:
#Credit: https://www.kaggle.com/parthsuresh/binary-classifier-using-keras-97-98-accuracy
# Simple modification made to code